In [1]:
# main imports
import re
import os
import sys
import ast
import copy
import glob
import math
import shutil
import random
import importlib
import subprocess

from itertools import product

import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats

import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import matplotlib.pylab as pylab
import upsetplot
import seaborn as sns

plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = 24
%matplotlib inline

pd.set_option('display.max_columns', None)

In [2]:
%load_ext autoreload
%autoreload 1

sys.path.insert(0, "/ccb/salz4-4/avaraby/orfanage/soft")
%aimport definitions

In [4]:
# paths

base_dir = "/ccb/salz4-4/avaraby/orfanage/"
data_dir = base_dir+"data/"

orfanage_bin = base_dir+"bin/orfanage"
gffcompare_bin = "/ccb/salz7-data/sw2/bin/gffcompare"
gffread_bin = "/ccb/salz7-data/sw2/bin/gffread"
igvtools_bin = "/ccb/salz7-data/sw/bin/igvtools"
sashimi_bin = base_dir+"bin/sashimi.py"

fa_fname = base_dir+"data/hg38.fa"
gtf_fname = base_dir+"data/MANE.v10.refseq.gtf"
mane_gtf_fname = base_dir+"data/MANE.v10.refseq.gtf"

outdir = base_dir+"longest_mane_rev1/"
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [5]:
# arguments
num_threads = 30

In [6]:
gtf_adjstop_fname = gtf_fname.rsplit(".",1)[0]+".adjstop.gtf"
gtf_adjstop_sorted_fname = gtf_adjstop_fname.rsplit(".",1)[0]+".sorted.gtf"
gtf_adjstop_aa_fa_fname = gtf_adjstop_fname.rsplit(".",1)[0]+".aa.fa"

clean_gtf_fname = gtf_adjstop_fname.rsplit(".",1)[0]+".clean.gtf"
nocds_gtf_fname = clean_gtf_fname.rsplit(".",1)[0]+".nocds.gtf"
nocds_gff_fname = nocds_gtf_fname.rsplit(".",1)[0]+".gff3"
nocds_fa_fname = nocds_gtf_fname.rsplit(".",1)[0]+".fa"

out_gtf_fname = outdir+"orf.gtf"
out_stats_fname = outdir+"orf.stats"
out_gtf_sorted_fname = outdir.rsplit(".",1)[0]+".sorted.gtf"

out_df_tsv_fname = outdir+"df.tsv"

In [7]:
# extract the longest ORF for each transcript
# run the same comparison as in transdecoder

In [8]:
# nocds_gtf_fname = data_dir+"longest.refseq.test.gtf"

In [10]:
# standardize the annotation - adjust stop, discard anything without a start/stop codon
cmd = [gffread_bin,
       "-g",fa_fname,
       "--adj-stop","-T","-F","-J",
       "-o",gtf_adjstop_fname,
       gtf_fname]

print(" ".join(cmd))
subprocess.call(cmd)

igv_cmd = [igvtools_bin,"sort",gtf_adjstop_fname,gtf_adjstop_sorted_fname]
print(" ".join(igv_cmd))
subprocess.call(igv_cmd)
igv_cmd = [igvtools_bin,"index",gtf_adjstop_sorted_fname]
print(" ".join(igv_cmd))
subprocess.call(igv_cmd)

cmd = [gffread_bin,
       "-y",gtf_adjstop_aa_fa_fname,
       "-g",fa_fname,
       gtf_adjstop_fname]

print(" ".join(cmd))
subprocess.call(cmd)

/ccb/salz7-data/sw2/bin/gffread -g /ccb/salz4-4/avaraby/orfanage/data/hg38.fa --adj-stop -T -F -J -o /ccb/salz4-4/avaraby/orfanage/data/MANE.v10.refseq.adjstop.gtf /ccb/salz4-4/avaraby/orfanage/data/MANE.v10.refseq.gtf
/ccb/salz7-data/sw/bin/igvtools sort /ccb/salz4-4/avaraby/orfanage/data/MANE.v10.refseq.adjstop.gtf /ccb/salz4-4/avaraby/orfanage/data/MANE.v10.refseq.adjstop.sorted.gtf
Sorting /ccb/salz4-4/avaraby/orfanage/data/MANE.v10.refseq.adjstop.gtf  -> /ccb/salz4-4/avaraby/orfanage/data/MANE.v10.refseq.adjstop.sorted.gtf
Done
/ccb/salz7-data/sw/bin/igvtools index /ccb/salz4-4/avaraby/orfanage/data/MANE.v10.refseq.adjstop.sorted.gtf
Done
/ccb/salz7-data/sw2/bin/gffread -y /ccb/salz4-4/avaraby/orfanage/data/MANE.v10.refseq.adjstop.aa.fa -g /ccb/salz4-4/avaraby/orfanage/data/hg38.fa /ccb/salz4-4/avaraby/orfanage/data/MANE.v10.refseq.adjstop.gtf


0

In [11]:
# get ids to remove
# 1. seleno
# 2. polycistronic
# 3. other

# polycistronic
df = definitions.get_chains(gtf_fname,"CDS",True)
df = df[df["has_cds"]==1].reset_index(drop=True)
df["seqid"]=df["coords"].str.split(":",n=1,expand=True)[0]
df["start"] = df["chain"].apply(lambda row: row[0][0])
df["end"] = df["chain"].apply(lambda row: row[-1][1])
# add gene ids
gid=pd.read_csv(gtf_fname,sep="\t",names=definitions.gff3cols,comment="#")
gid=gid[gid["type"]=="transcript"].reset_index(drop=True)
gid["tid"]=gid["attributes"].str.split("transcript_id \"",expand=True)[1].str.split("\"",expand=True)[0]
gid["gid"]=gid["attributes"].str.split("gene_id \"",expand=True)[1].str.split("\"",expand=True)[0]
gid = gid[["gid","tid"]]
print("total number of genes: "+str(len(set(gid["gid"]))))

df = df.merge(gid,on="tid",how="left",indicator=False)

print("total number of coding genes: "+str(len(set(df["gid"]))))

df["start"] = df["start"].astype(int)
df["end"] = df["end"].astype(int)

df.sort_values(by=["seqid","strand","start","end"],ascending=True,inplace=True)

df = df.groupby(by=["seqid","strand","gid"]).agg({"start":min,"end":max}).reset_index()
df.sort_values(by=["seqid","strand","start","end"],ascending=True,inplace=True)
df["nc"]=df.seqid.shift(-1)
df["nt"]=df.strand.shift(-1)
df["ns"]=df.start.shift(-1)
df["nid"]=df.gid.shift(-1)
df.fillna(0,inplace=True)
df["od"] = np.where((df["seqid"]==df["nc"]) & 
                           (df["strand"]==df["nt"]) & 
                           (df["end"]>df["ns"]),1,0)
pids = set(df[df["od"]==1]["gid"]).union(set(df[df["od"]==1]["nid"]))
print("number of poly: "+str(len(pids)))

# seleno and other exceptions

df=pd.read_csv(gtf_fname,sep="\t",names=definitions.gff3cols,comment="#")
df=df[df["type"]=="transcript"].reset_index(drop=True)
df["tid"]=df["attributes"].str.split("transcript_id \"",expand=True)[1].str.split("\"",expand=True)[0]
df["gid"]=df["attributes"].str.split("gene_id \"",expand=True)[1].str.split("\"",expand=True)[0]
df["attr"]=df["attributes"].str.split("exception \"",expand=True)[1].str.split("\"",expand=True)[0]
df["seleno"] = df["attributes"].str.lower().str.contains("selen")

sids = set(df[df["seleno"]]["gid"])
print("number of seleno: "+str(len(sids)))

print("exceptions: "+", ".join(list(set(df[~(df["attr"].isna())]["attr"].tolist()))))
eids = set(df[~(df["attr"].isna())]["gid"])
print("number of exceptions: "+str(len(eids)))

total number of genes: 19062
total number of coding genes: 19062
number of poly: 142
number of seleno: 32
exceptions: dicistronic gene, alternative start codon
number of exceptions: 18


In [12]:
dirty_gids = pids.union(sids).union(eids)
print("number of genes to discard: "+str(len(dirty_gids)))

number of genes to discard: 189


In [13]:
# create a file without transcripts in these genes

with open(clean_gtf_fname,"w+") as outFP:
    with open(gtf_adjstop_fname,"r") as inFP:
        for line in inFP:
            lcs = line.split("\t")
            gid = lcs[8].split("gene_id \"",1)[1].split("\"",1)[0]
            if not gid in dirty_gids:
                outFP.write(line)

In [14]:
# create input without CDS in it (easier to explain than "not using --keep_cds")

with open(nocds_gtf_fname,"w+") as outFP:
    with open(clean_gtf_fname,"r") as inFP:
        for line in inFP:
            lcs = line.split("\t")
            if lcs[2]=="CDS":
                continue
            
            outFP.write(line)

In [15]:
# run orfanage with mane as the reference
cmd = [orfanage_bin,
       "--reference",fa_fname,
       "--query",nocds_gtf_fname,
       "--threads",str(num_threads),
       "--output",out_gtf_fname,
       "--stats",out_stats_fname,
       mane_gtf_fname]
print(" ".join(cmd))
subprocess.call(cmd)

/ccb/salz4-4/avaraby/orfanage/bin/orfanage --reference /ccb/salz4-4/avaraby/orfanage/data/hg38.fa --query /ccb/salz4-4/avaraby/orfanage/data/MANE.v10.refseq.adjstop.clean.nocds.gtf --threads 30 --output /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/orf.gtf --stats /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/orf.stats /ccb/salz4-4/avaraby/orfanage/data/MANE.v10.refseq.gtf


loading reference genome
loading reference transcriptomes
sorting reference transcriptome
loading query transcriptome
bundling transcriptome
starting main evaluation


0

In [16]:
cmd = [gffread_bin,"-T","-o",out_gtf_fname.split(".gtf")[0]+".gffread.gtf",out_gtf_fname]
print(" ".join(cmd))
subprocess.call(cmd)
igv_cmd = [igvtools_bin,"sort",out_gtf_fname.split(".gtf")[0]+".gffread.gtf",out_gtf_fname.split(".gtf")[0]+".sorted.gtf"]
print(" ".join(igv_cmd))
subprocess.call(igv_cmd)
igv_cmd = [igvtools_bin,"index",out_gtf_fname.split(".gtf")[0]+".sorted.gtf"]
print(" ".join(igv_cmd))
subprocess.call(igv_cmd)

/ccb/salz7-data/sw2/bin/gffread -T -o /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/orf.gffread.gtf /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/orf.gtf
/ccb/salz7-data/sw/bin/igvtools sort /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/orf.gffread.gtf /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/orf.sorted.gtf
Sorting /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/orf.gffread.gtf  -> /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/orf.sorted.gtf
Done
/ccb/salz7-data/sw/bin/igvtools index /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/orf.sorted.gtf
Done


0

In [17]:
# extract nt fasta with gffread
cmd = [gffread_bin,
       "-g",fa_fname,
       "-w",outdir+"tx.nt.fasta",
       nocds_gtf_fname]
print(" ".join(cmd))
subprocess.call(cmd)

/ccb/salz7-data/sw2/bin/gffread -g /ccb/salz4-4/avaraby/orfanage/data/hg38.fa -w /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/tx.nt.fasta /ccb/salz4-4/avaraby/orfanage/data/MANE.v10.refseq.adjstop.clean.nocds.gtf


0

In [ ]:
tx_nt_dict = definitions.load_fasta_dict(outdir+"tx.nt.fasta",False,True)
chains = definitions.get_chains(nocds_gtf_fname,"exon",True)

In [22]:
# find longest ORF in each transcript
# what do we do if there is multiple ORFs of the same length? - just count for now. could also just skip those genes alltogether
def find_longest_orfs(seq):
    longest = []
    max_len = 0

    matches = re.finditer(r'(?=(ATG(?:(?!TAA|TAG|TGA)...)*(?:TAA|TAG|TGA)))', seq)
    for match in matches:
        result = match.group(1)
        coords = [match.start(),match.start()+len(result)-1]

        if max_len<len(result):
            longest = [coords]
            max_len = len(result)
            continue
        if max_len==len(result):
            longest.append(coords)
            continue

    return longest

def trans2genome(chain, strand, zero_pos):
    chain_pos = -1
    left_to_stop = zero_pos
    found_pos = False
    if strand=='+':
        for i in range(len(chain)):
            clen = definitions.slen(chain[i])
            if left_to_stop<clen: # found the segment with the stop codon
                chain_pos = chain[i][0]+left_to_stop
                found_pos = True
                break
            
            left_to_stop-=clen
        
        if not found_pos: # return the last position
            chain_pos = chain[-1][1]
        
    else:
        for i in range(len(chain)-1,-1,-1):
            clen = definitions.slen(chain[i])
            if left_to_stop<clen: # found the cds segment with the stop codon
                chain_pos = chain[i][1]-left_to_stop
                found_pos = True
                break
            
            left_to_stop-=clen
            
        if not found_pos: # return the last position
            chain_pos = chain[0][0]
        
    assert chain_pos>=0,"unexpected chain_pos<0"
    return chain_pos

def cut_chain(chain, start, end):
    res = []
    for cs,ce in chain:
        new_cs = cs
        new_ce = ce
        if new_cs<=start and new_ce>=start:
            new_cs = start
        if new_ce>=end:
            new_ce = end
            res.append([new_cs,new_ce])
            break
        if new_ce<start or new_cs>end:
            continue
        res.append([new_cs,new_ce])
    return res

In [23]:
orfs = dict()

for tid,tx_nt in tx_nt_dict.items():
    chain = chains[chains["tid"]==tid].iloc[0].chain
    strand = chains[chains["tid"]==tid].iloc[0].strand
    
    torfs = find_longest_orfs(tx_nt)
    if torfs is None or len(torfs)==0:
        continue
    
    if len(torfs)>1: # skip for now
        print("multiple longest ORFs found: "+tid)        
        
    for torf in torfs:
        gstart = trans2genome(chain,strand,torf[0])
        gend = trans2genome(chain,strand,torf[1])

        orf = cut_chain(chain,min(gstart,gend),max(gstart,gend))
        orfs.setdefault(tid,list())
        orfs[tid].append(orf)

multiple longest ORFs found: rna-NM_176823.4
multiple longest ORFs found: rna-NM_004137.4
multiple longest ORFs found: rna-NM_144589.4
multiple longest ORFs found: rna-NM_001164257.2
multiple longest ORFs found: rna-NM_017914.4


In [24]:
# now we just need ot output results into a GTF
multi_orfs = set()
with open(outdir+"longest.gtf","w+") as outFP:
    with open(nocds_gtf_fname,"r") as inFP:
        for line in inFP:
                
            lcs = line.strip().split('\t')
            
            if lcs[2] in ["transcript","exon"]:
                outFP.write(line)
            
            if lcs[2] == "transcript":
                tid = lcs[8].split("transcript_id \"", 1)[1].split("\"", 1)[0]
                if not tid in orfs:
                    continue
                if len(orfs[tid])>1:
                    print("multiple orfs: "+tid)
                    multi_orfs.add(tid)
                    continue
                    
                cds_lcs = copy.deepcopy(lcs)
                cds_lcs[2] = "CDS"
                cds_lcs[8] = "transcript_id \""+tid+"\";"
                
                for orf in orfs[tid]:
                    for cs,ce in orf:
                        cds_lcs[3] = str(int(cs))
                        cds_lcs[4] = str(int(ce))
                        outFP.write("\t".join(cds_lcs)+"\n")

multiple orfs: rna-NM_176823.4
multiple orfs: rna-NM_004137.4
multiple orfs: rna-NM_144589.4
multiple orfs: rna-NM_001164257.2
multiple orfs: rna-NM_017914.4


In [25]:
cmd = [gffread_bin,"-T","-o",outdir+"longest.gffread.gtf",outdir+"longest.gtf"]
print(" ".join(cmd))
subprocess.call(cmd)
igv_cmd = [igvtools_bin,"sort",outdir+"longest.gffread.gtf",outdir+"longest.sorted.gtf"]
print(" ".join(igv_cmd))
subprocess.call(igv_cmd)
igv_cmd = [igvtools_bin,"index",outdir+"longest.sorted.gtf"]
print(" ".join(igv_cmd))
subprocess.call(igv_cmd)

/ccb/salz7-data/sw2/bin/gffread -T -o /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/longest.gffread.gtf /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/longest.gtf
/ccb/salz7-data/sw/bin/igvtools sort /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/longest.gffread.gtf /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/longest.sorted.gtf
Sorting /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/longest.gffread.gtf  -> /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/longest.sorted.gtf
Done
/ccb/salz7-data/sw/bin/igvtools index /ccb/salz4-4/avaraby/orfanage/longest_mane_rev1/longest.sorted.gtf
Done


0

In [39]:
longest_ne_def = df[~(df["def_chain"]==df["lng_chain"])].reset_index(drop=True)
print("number of MANE transcripts for which the longest ORF is not the annotated one: "+str(len(longest_ne_def)))

number of MANE transcripts for which the longest ORF is not the annotated one: 568
